In [36]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json
import pprint
import requests

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token


In [37]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [38]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]


In [39]:
def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    json_result=json.loads(result.content)
    return json_result['tracks']

In [40]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    json_result=json.loads(result.content)["tracks"]
    return json_result

In [41]:

token = get_token()
print(token)

BQAhsNTvtyBNO7pbUcI3jWgIz94fO5eMMz-JJ1v1zHUQCE4XrGcJbt_r7fmKAYwI2hGPunartWWu10tuqYug5xunW3YE334-bZ_tPUe9U-Y5qIsC-8g


In [42]:
artist_input = input("What artist are you looking for")




In [43]:
popularity = int(input("How popular do you want the recommendations to be? (0-100)"))

if popularity <10:
    min_popularty = 0
    max_popularity = popularity +10
elif popularity >90:
    min_popularty =popularity -10
    max_popularity = 100
else:
    min_popularty= popularity - 10
    max_popularity = popularity +10
    
print(min_popularty, max_popularity)


10 30


In [44]:
result = search_for_artist(token, artist_input)
artist_id = result["id"]
# songs = get_songs_recommendations(token, artist_id, min_popularty, max_popularity)
songs = []

recommended_song_list = []
recommended_song_id = []
song_low =  get_songs_recommendations(token, artist_id, 0, 33)
song_med =  get_songs_recommendations(token, artist_id, 34, 66)
song_high =  get_songs_recommendations(token, artist_id, 67, 100)

songs.extend(song_low)
songs.extend(song_med)


In [45]:
songs

[{'album': {'album_type': 'ALBUM',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4pVWSc4qEC6U3yIjQleh90'},
     'href': 'https://api.spotify.com/v1/artists/4pVWSc4qEC6U3yIjQleh90',
     'id': '4pVWSc4qEC6U3yIjQleh90',
     'name': 'Trash Talk',
     'type': 'artist',
     'uri': 'spotify:artist:4pVWSc4qEC6U3yIjQleh90'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/3q81wYkuErRE4zeihVDeGu'},
   'href': 'https://api.spotify.com/v1/albums/3q81wYkuErRE4zeihVDeGu',
   'id': '3q81wYkuErRE4zeihVDeGu',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273a08fe7f446078e36f386dd06',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02a08fe7f446078e36f386dd06',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851a08fe7f446078e36f386dd06',
     'width': 64}],
   'is_playable': True,
   'name': '119',
   'release_date': '2012-10-09',
   'r

In [46]:
for song in songs:
    # print(song['name'])
    # print(song['id'])
    recommended_song_id.append(song['id'])
    for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
        dict = {
            'name' : song['name'],
            'artist' : artist['name'],
            'id': song['id'],
            'popularity' : song['popularity'],

        }
        recommended_song_list.append(dict)


print(recommended_song_list)
print(recommended_song_id)

[{'name': 'Blossom & Burn (feat. Hodgy & Tyler, The Creator)', 'artist': 'Trash Talk', 'id': '1hIIXXrgAExOVgkMcIipI4', 'popularity': 26}, {'name': 'Blossom & Burn (feat. Hodgy & Tyler, The Creator)', 'artist': 'Hodgy', 'id': '1hIIXXrgAExOVgkMcIipI4', 'popularity': 26}, {'name': 'Blossom & Burn (feat. Hodgy & Tyler, The Creator)', 'artist': 'Tyler, The Creator', 'id': '1hIIXXrgAExOVgkMcIipI4', 'popularity': 26}, {'name': 'Go (Gas) (feat. Wiz Khalifa, Juicy J & Tyler, The Creator)', 'artist': 'Domo Genesis', 'id': '0v3IeMw8l2M3TjIqQrihDW', 'popularity': 32}, {'name': 'Go (Gas) (feat. Wiz Khalifa, Juicy J & Tyler, The Creator)', 'artist': 'Wiz Khalifa', 'id': '0v3IeMw8l2M3TjIqQrihDW', 'popularity': 32}, {'name': 'Go (Gas) (feat. Wiz Khalifa, Juicy J & Tyler, The Creator)', 'artist': 'Juicy J', 'id': '0v3IeMw8l2M3TjIqQrihDW', 'popularity': 32}, {'name': 'Go (Gas) (feat. Wiz Khalifa, Juicy J & Tyler, The Creator)', 'artist': 'Tyler, The Creator', 'id': '0v3IeMw8l2M3TjIqQrihDW', 'popularity'

In [47]:
top_songs_by_artist = get_songs_by_artist(token, artist_id)

song_list = []
song_id = []

for song in top_songs_by_artist:
    # print(song['name'])
    # print(song['id'])
    song_id.append(song['id'])
    for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
        dict = {
            'name' : song['name'],
            'artist' : artist['name'],
            'id': song['id'],
            'popularity' : song['popularity']
        }
        song_list.append(dict)


print(song_list)
print(song_id)

[{'name': 'See You Again (feat. Kali Uchis)', 'artist': 'Tyler, The Creator', 'id': '7KA4W4McWYRpgf0fWsJZWB', 'popularity': 93}, {'name': 'See You Again (feat. Kali Uchis)', 'artist': 'Kali Uchis', 'id': '7KA4W4McWYRpgf0fWsJZWB', 'popularity': 93}, {'name': 'NEW MAGIC WAND', 'artist': 'Tyler, The Creator', 'id': '0fv2KH6hac06J86hBUTcSf', 'popularity': 83}, {'name': 'ARE WE STILL FRIENDS?', 'artist': 'Tyler, The Creator', 'id': '5TxRUOsGeWeRl3xOML59Ai', 'popularity': 82}, {'name': 'BEST INTEREST', 'artist': 'Tyler, The Creator', 'id': '3jHdKaLCkuNEkWcLVmQPCX', 'popularity': 82}, {'name': 'EARFQUAKE', 'artist': 'Tyler, The Creator', 'id': '5hVghJ4KaYES3BFUATCYn0', 'popularity': 82}, {'name': 'She', 'artist': 'Tyler, The Creator', 'id': '0SA0mMXWh23raZ6xzOCU2V', 'popularity': 77}, {'name': 'She', 'artist': 'Frank Ocean', 'id': '0SA0mMXWh23raZ6xzOCU2V', 'popularity': 77}, {'name': 'SWEET / I THOUGHT YOU WANTED TO DANCE (feat. Brent Faiyaz & Fana Hues)', 'artist': 'Tyler, The Creator', 'id'

In [48]:
def get_song_audio_features(token, song_id):
    url = f"https://api.spotify.com/v1/audio-features?ids={','.join(song_id)}"
    headers = get_auth_header(token)
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        json_result = response.json()  # Use .json() method to parse the JSON data
        return json_result
    else:
        print(f"Failed to fetch audio features. Status code: {response.status_code}")
        return None

    return(json_result)

In [49]:
recommended_features = get_song_audio_features(token, recommended_song_id)
top_artist_features = get_song_audio_features(token, song_id)
print(recommended_features['audio_features'])

Failed to fetch audio features. Status code: 400


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
df_recommended_audiofeatures =pd.DataFrame(recommended_features['audio_features'])

df_top_artist_audiofeatures = pd.DataFrame(top_artist_features['audio_features'])


In [ ]:

df_recommended_song_list = pd.DataFrame(recommended_song_list)
df_recommended_song_list = df_recommended_song_list.drop_duplicates(subset=['name']).reset_index(drop=True)

df_top_artist_song_list = pd.DataFrame(song_list)
df_top_artist_song_list = df_top_artist_song_list.drop_duplicates(subset=['name']).reset_index(drop=True)

In [ ]:
recommendations_df = pd.merge(df_recommended_song_list, df_recommended_audiofeatures, on ='id')

recommendations_df['duration_s'] = recommendations_df['duration_ms'] / 1000
recommendations_df

,name,artist,id,popularity,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_s
0,Adore You,Harry Styles,3jjujdWJ72nww5eGnfs2E7,88,0.676,0.771,8,-3.675,1,0.0483,...,0.1020,0.5690,99.048,audio_features,spotify:track:3jjujdWJ72nww5eGnfs2E7,https://api.spotify.com/v1/tracks/3jjujdWJ72nw...,https://api.spotify.com/v1/audio-analysis/3jju...,207133,4,207.133
1,Left and Right (Feat. Jung Kook of BTS),Charlie Puth,5Odq8ohlgIbQKMZivbWkEo,88,0.881,0.592,2,-4.898,1,0.0324,...,0.0901,0.7190,101.058,audio_features,spotify:track:5Odq8ohlgIbQKMZivbWkEo,https://api.spotify.com/v1/tracks/5Odq8ohlgIbQ...,https://api.spotify.com/v1/audio-analysis/5Odq...,154487,4,154.487
2,Lose Somebody,Kygo,1sgDyuLooyvEML4oHspNza,74,0.580,0.586,1,-6.883,1,0.0357,...,0.0755,0.5070,147.988,audio_features,spotify:track:1sgDyuLooyvEML4oHspNza,https://api.spotify.com/v1/tracks/1sgDyuLooyvE...,https://api.spotify.com/v1/audio-analysis/1sgD...,199549,4,199.549
3,hate u love u,Olivia O'Brien,6ATgDc6e4sPn84hvJsAmPt,74,0.407,0.416,6,-8.915,0,0.1740,...,0.0981,0.2460,176.947,audio_features,spotify:track:6ATgDc6e4sPn84hvJsAmPt,https://api.spotify.com/v1/tracks/6ATgDc6e4sPn...,https://api.spotify.com/v1/audio-analysis/6ATg...,175467,4,175.467
4,Here With Me,Marshmello,5icOoE6VgqFKohjWWNp0Ac,77,0.791,0.565,5,-3.933,0,0.0439,...,0.1560,0.1810,99.961,audio_features,spotify:track:5icOoE6VgqFKohjWWNp0Ac,https://api.spotify.com/v1/tracks/5icOoE6VgqFK...,https://api.spotify.com/v1/audio-analysis/5icO...,156347,4,156.347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,I Love You 3000,Stephanie Poetri,3znQ9i61vfe2E7URHlOiyc,74,0.648,0.382,6,-7.850,1,0.0396,...,0.3930,0.4080,90.099,audio_features,spotify:track:3znQ9i61vfe2E7URHlOiyc,https://api.spotify.com/v1/tracks/3znQ9i61vfe2...,https://api.spotify.com/v1/audio-analysis/3znQ...,209734,4,209.734
95,I Like Me Better,Lauv,0EcQcdcbQeVJn9fknj44Be,85,0.753,0.505,9,-7.621,1,0.2550,...,0.1040,0.4190,91.972,audio_features,spotify:track:0EcQcdcbQeVJn9fknj44Be,https://api.spotify.com/v1/tracks/0EcQcdcbQeVJ...,https://api.spotify.com/v1/audio-analysis/0EcQ...,197437,4,197.437
96,"Don't Give Up On Me - (From ""Five Feet Apart"")",Andy Grammer,5A3fPy30SN2wuzrahpcxvV,75,0.641,0.537,0,-5.452,0,0.0282,...,0.1150,0.0681,112.864,audio_features,spotify:track:5A3fPy30SN2wuzrahpcxvV,https://api.spotify.com/v1/tracks/5A3fPy30SN2w...,https://api.spotify.com/v1/audio-analysis/5A3f...,196188,4,196.188
97,"thank u, next",Ariana Grande,3e9HZxeyfWwjeyPAMmWSSQ,84,0.717,0.653,1,-5.634,1,0.0658,...,0.1010,0.4120,106.966,audio_features,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4,207.320


In [ ]:
top_artist_df = pd.merge(df_top_artist_song_list, df_top_artist_audiofeatures, on ='id')
top_artist_df['duration_s'] = top_artist_df['duration_ms'] / 1000
top_artist_df

,name,artist,id,popularity,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_s
0,I Like Me Better,Lauv,0EcQcdcbQeVJn9fknj44Be,85,0.753,0.505,9,-7.621,1,0.2550,...,0.1040,0.4190,91.972,audio_features,spotify:track:0EcQcdcbQeVJn9fknj44Be,https://api.spotify.com/v1/tracks/0EcQcdcbQeVJ...,https://api.spotify.com/v1/audio-analysis/0EcQ...,197437,4,197.437
1,"Steal The Show - From ""Elemental""",Lauv,6R5fYCySNHrqo4Og6O1ppn,78,0.620,0.640,7,-7.308,1,0.0305,...,0.1400,0.4090,92.522,audio_features,spotify:track:6R5fYCySNHrqo4Og6O1ppn,https://api.spotify.com/v1/tracks/6R5fYCySNHrq...,https://api.spotify.com/v1/audio-analysis/6R5f...,191779,4,191.779
2,i'm so tired...,Lauv,7LpMG3AEaIJgs62HQhAa91,78,0.548,0.735,11,-7.572,1,0.2360,...,0.1950,0.4980,101.252,audio_features,spotify:track:7LpMG3AEaIJgs62HQhAa91,https://api.spotify.com/v1/tracks/7LpMG3AEaIJg...,https://api.spotify.com/v1/audio-analysis/7LpM...,162587,4,162.587
3,Who,Lauv,0pSqYDTjY1Xt86usTQslAx,76,0.632,0.337,11,-8.000,0,0.0330,...,0.1040,0.0982,141.701,audio_features,spotify:track:0pSqYDTjY1Xt86usTQslAx,https://api.spotify.com/v1/tracks/0pSqYDTjY1Xt...,https://api.spotify.com/v1/audio-analysis/0pSq...,180413,3,180.413
4,Paris in the Rain,Lauv,2MOoIbJWIYikwIXjBDe26i,75,0.406,0.408,3,-9.419,1,0.0530,...,0.3940,0.5250,79.705,audio_features,spotify:track:2MOoIbJWIYikwIXjBDe26i,https://api.spotify.com/v1/tracks/2MOoIbJWIYik...,https://api.spotify.com/v1/audio-analysis/2MOo...,204712,4,204.712
5,All 4 Nothing (I'm So In Love),Lauv,7I5WT7DtpwSAhEeDLpZEfm,71,0.720,0.819,1,-4.588,1,0.0362,...,0.0606,0.8980,131.990,audio_features,spotify:track:7I5WT7DtpwSAhEeDLpZEfm,https://api.spotify.com/v1/tracks/7I5WT7DtpwSA...,https://api.spotify.com/v1/audio-analysis/7I5W...,183014,4,183.014
6,Never Not,Lauv,3ZPBZH2k9p306C1bu7bkhC,74,0.471,0.220,5,-9.243,1,0.0370,...,0.1040,0.2600,133.714,audio_features,spotify:track:3ZPBZH2k9p306C1bu7bkhC,https://api.spotify.com/v1/tracks/3ZPBZH2k9p30...,https://api.spotify.com/v1/audio-analysis/3ZPB...,208000,4,208.000
7,Feelings,Lauv,6r3duEAfFTH83DuoywkG20,73,0.700,0.394,9,-8.911,1,0.0328,...,0.0801,0.2410,103.041,audio_features,spotify:track:6r3duEAfFTH83DuoywkG20,https://api.spotify.com/v1/tracks/6r3duEAfFTH8...,https://api.spotify.com/v1/audio-analysis/6r3d...,189907,4,189.907
8,Mean It,Lauv,6mXdCcFnPKQznj4CmMRmHC,72,0.747,0.450,7,-8.543,1,0.0919,...,0.1700,0.3290,96.001,audio_features,spotify:track:6mXdCcFnPKQznj4CmMRmHC,https://api.spotify.com/v1/tracks/6mXdCcFnPKQz...,https://api.spotify.com/v1/audio-analysis/6mXd...,232853,4,232.853
9,"fuck, i'm lonely",Lauv,2RNsweBBb1a6tAHXQtt2xH,70,0.809,0.547,9,-7.305,1,0.0614,...,0.0540,0.7180,94.992,audio_features,spotify:track:2RNsweBBb1a6tAHXQtt2xH,https://api.spotify.com/v1/tracks/2RNsweBBb1a6...,https://api.spotify.com/v1/audio-analysis/2RNs...,198973,4,198.973
